# Activity: Let's Break the Student Record Application
There is a big difference between writing code that only you will use and writing code that others will use. When you write code for others, you need to make sure that it is easy to use,and robust enough to handle unexpected inputs. 

In this activity, you will be given a code base, in particular the `build(...)` and `find(...)` methods from the student records app, that is not robust and has some bugs. Some of these bugs ae can fix, while others are just errors in user logic. 

Our task is to break the code by providing unexpected inputs, look at how the system reacts (who is doing what) and then suggest fixes to make it more robust for items that we can control. Let's go!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.
* The [include command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 
* In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl), check out that documentation for more information on the functions and types used in this material.

In [1]:
include("Include.jl");

### Types and Functions
Let's start by defining mutable `MySimpleStudentModel` type, which will represent a student record. This type will include fields for the student's first name, last name, student identification number, and thier netid (email). 
* _Why a mutable type?_ Mutable types allow for modification of their fields after creation, which is useful for objects that may need to be updated or changed over time, such as student records. Additonally, mutable types offer different possible initialization options, such as default values for fields.
* _Keyword argument constructor_: The constructor allows for the creation of `MySimpleStudentModel` objects with default values for fields, making it easier to instantiate objects without providing all arguments. Thus, we can create an empty student record with default values for each field.

In [2]:
"""
    mutable struct MySimpleStudentModel

A mutable struct that models a student with a firstname, lastname, student id and a netid.

### Fields
- `firstname::String`: The first name of the student.
- `lastname::String`: The last name of the student.
- `sid::Int64`: The student identification number.
- `netid::String`: The network identifier (email address) of the student.
"""
mutable struct MySimpleStudentModel

    # data fields -
    firstname::String
    lastname::String
    sid::Int64
    netid::String
    
    # keyword argument constructor: builds a new student model with default values
    MySimpleStudentModel(; firstname::String = "firstname", 
        lastname::String = "lastname", sid::Int64 = -1, netid::String = "abcd") = new(firstname, lastname, sid, netid);
end;

Let's implement a `build(model::Type{MySimpleStudentModel}; data::NamedTuple)::MyStudentModel` method which takes the type of thing we want to build, and the data need to build the model [in a `NamedTuple` instance](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple). The `build(...)` returns a populated `MySimpleStudentModel` instance.

In [3]:
"""
    build(modeltype::Type{MySimpleStudentModel}, data::NamedTuple)::MySimpleStudentModel

Builds a new `MySimpleStudentModel` from a named tuple of data.

### Arguments
- `modeltype::Type{MySimpleStudentModel}`: The type of the model to be built.
- `data::NamedTuple`: A named tuple containing the fields to be set in the model.

### Returns
- `MySimpleStudentModel`: A new instance of `MySimpleStudentModel` with the fields set from the named tuple.
"""
function build(modeltype::Type{MySimpleStudentModel}, data::NamedTuple)::MySimpleStudentModel
    
    # initailize -
    model = modeltype(); # This builds an empty model (with default values)

    # TODO: Unommment below for added default values if needed
    # get(data, :firstname, "default_firstname");
    # get(data, :lastname, "default_lastname");
    # get(data, :sid, -1);
    # get(data, :netid, "default_netid");

    # TODO: Unomment the following lines to set the fields of the model
    # get data from the named tuple and set the fields of the model
    model.firstname = data.firstname;
    model.lastname = data.lastname;
    model.sid = data.sid;
    model.netid = data.netid;
    
    # return -
    return model;
end;

Finally, we have the implementation of a [`find(...)` method](src/Compute.jl). This method takes a collection of student models and values for the fields of the student we want and returns either the student index `sid::Int64` matching the other search fields or `nothing`.

In [4]:
"""
    find(students::Array{MySimpleStudentModel,1}; netid::String="jdv27", firstname::String = "firstname", 
        lastname::String = "lastname") -> Union{Int64, Nothing}

Finds a student in the array of `MySimpleStudentModel` based on the provided parameters. 
Returns the student's `sid` if found, otherwise returns `nothing`.

### Parameters
- `students::Array{MySimpleStudentModel,1}`: An array of `MySimpleStudentModel` objects.
- `netid::String`: The netid of the student to search for (default is "jdv27").
- `firstname::String`: The firstname of the student to search for (default is "firstname").
- `lastname::String`: The lastname of the student to search for (default is "lastname").

### Returns
- `Union{Int64, Nothing}`: The student's `sid` if found, otherwise `nothing`.
"""
function find(students::Array{MySimpleStudentModel,1}; netid::String="jdv27", firstname::String = "firstname", 
    lastname::String = "lastname")::Union{Int64, Nothing}

    # initialize -
    student_index = nothing; # default: we don't know which student we are looking for

    # main loop -
    for i ∈ eachindex(students)
        test_student = students[i];  # get student i from the array -

        # TODO: Uncomment the if block below to check with compound AND
        # if statement: does test_student have the same netid, firstname and lastname as the parameters?
        # if test_student.netid == netid && test_student.firstname == firstname && test_student.lastname == lastname
        #     student_index = test_student.sid; # we found the student, so set the index to the student's sid
        #     break; # exit the loop
        # end

        # TODO: Uncomment the if block below to check with compound OR
        if test_student.netid == netid || test_student.firstname == firstname || test_student.lastname == lastname
            student_index = test_student.sid; # we found the student, so set the index to the student's sid
            break; # exit the loop
        end
    end

    return student_index; # return the search results to the caller
end;

## Task 1: Break the Build Method
In this task, let's try to break the `build(...)` method by providing unexpected inputs. For example, we'll provide an incorrect type of the object we want to build, and will provide a `data::NamedTuple` with missing or incorrect fields and see how the system reacts.

The intersting bit of the experiment is to see who is doing what when the system reacts. For example, are the errors things that we could anticipate and handle, or are they errors in whcih the system if responsible for handling? Does the system throw an error, or does it return a default value? Does it provide a helpful error message, or is it cryptic?

### Incorrect Object Type
The first argument to the `build(...)` method is the type of the object we want to build. Let's try to provide an incorrect type, such as `Int64` instead of `MySimpleStudentModel`. 

* _What do you expect to happen_? This bug (which is a user logic error) should be caught by the system, and it should throw an error. The user is asking for a method with incorrect parameters in the type sense, so the system should not be able to find a matching method, and thus throw an error.

Let's see how the system reacts. Is this what happens?

In [5]:
build(Int64, (firstname = "John", lastname = "Doe", sid = 123456, netid = "jbd123")) # Example usage

MethodError: MethodError: no method matching build(::Type{Int64}, ::@NamedTuple{firstname::String, lastname::String, sid::Int64, netid::String})
The function `build` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  build(!Matched::Type{MySimpleStudentModel}, ::NamedTuple)
   @ Main ~/Desktop/julia_work/CHEME-140-eCornell-Repository/CHEME-140-eCornell-Repository/courses/CHEME-141/module-2/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X16sZmlsZQ==.jl:13
